# AOJ GRL_6_A

最大流問題

http://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=GRL_6_A&lang=jp

In [24]:
V, E = 4, 5
edges = [[0, 1, 2], [0, 2, 1], [1, 2, 1], [1, 3, 1], [2, 3, 2]]

print('ans: 3')

ans: 3


In [28]:
graph = [[] for _ in range(V)]
for u, v, c in edges:
    graph[u].append([v, c, len(graph[v])])
    graph[v].append([u, 0, len(graph[u]) - 1])
    
graph

[[[1, 2, 0], [2, 1, 0]],
 [[0, 0, 0], [2, 1, 1], [3, 1, 0]],
 [[0, 0, 1], [1, 0, 1], [3, 2, 1]],
 [[1, 0, 2], [2, 0, 2]]]

### Ford-Fulkerson algorithm

計算量は，最大流量をFとすると$O(F|E|)$

In [25]:
def dfs(u, t, f):
    if u == t:
        return f
    
    used[u] = True
    for i in range(len(graph[u])):
        v, c, r = graph[u][i]
        if (not used[v]) and c > 0:
            d = dfs(v, t, min(f, c))
            if d > 0:
                graph[u][i][1] -= d
                graph[v][r][1] += d
                return d

    return 0

In [26]:
graph = [[] for _ in range(V)]
for u, v, c in edges:
    graph[u].append([v, c, len(graph[v])])
    graph[v].append([u, 0, len(graph[u]) - 1])

s = 0
t = V - 1
flow = 0

while True:
    used = [False] * V
    f = dfs(s, t, float('inf'))
    if f == 0:
        break
    flow += f
    
print(flow)

3


### Dinic algorithm

In [31]:
from collections import deque

def bfs(s):
    level[s] = 0
    q = deque()
    q.append(s)
    
    while q:
        u = q.popleft()
        for v, c, _ in graph[u]:
            if c > 0 and level[v] < 0:
                level[v] = level[u] + 1
                q.append(v)

In [32]:
def dfs(u, t, f):
    if u == t:
        return f
            
    for i in range(len(graph[u])):
        v, c, r = graph[u][i]
        if c > 0 and level[u] < level[v]:
            d = dfs(v, t, min(f, c))
            if d > 0:
                graph[u][i][1] -= d
                graph[v][r][1] += d
                return d
                
    return 0

In [33]:
graph = [[] for _ in range(V)]
for u, v, c in edges:
    graph[u].append([v, c, len(graph[v])])
    graph[v].append([u, 0, len(graph[u]) - 1])

s = 0
t = V - 1
flow = 0

while True:
    level = [-1] * V
    bfs(s)
    if level[t] < 0:
        break
        
    visited = [False] * V
    while True:
        f = dfs(s, t, float('inf'))
        if f > 0:
            flow += f
        else:
            break
            
print(flow)

3


# ABC 10 - D

最小カット問題

https://atcoder.jp/contests/abc010/tasks/abc010_4

In [1]:
N, G, E = 4, 2, 3
p = [2, 3]
edges = [[0, 1], [1, 2], [1, 3]]

print('ans: 1')

ans: 1


#### 無向グラフの最大流問題

In [5]:
def gen_graph():
    graph = [[] for _ in range(N + 1)]
    for u, v in edges:
        graph[u].append([v, 1, len(graph[v])])
        graph[v].append([u, 0, len(graph[u]) - 1])
        
        graph[v].append([u, 1, len(graph[u])])
        graph[u].append([v, 0, len(graph[v]) - 1])

    for u in p:
        graph[u].append([N, 1, len(graph[N])])
        graph[N].append([u, 0, len(graph[u]) - 1])

    return graph
        
gen_graph()

[[[1, 1, 0], [1, 0, 1]],
 [[0, 0, 0], [0, 1, 1], [2, 1, 0], [2, 0, 1], [3, 1, 0], [3, 0, 1]],
 [[1, 0, 2], [1, 1, 3], [4, 1, 0]],
 [[1, 0, 4], [1, 1, 5], [4, 1, 1]],
 [[2, 0, 2], [3, 0, 2]]]

In [6]:
def dfs(u, t, f):
    if u == t:
        return f
    
    used[u] = True
    for i in range(len(graph[u])):
        v, c, r = graph[u][i]
        if (not used[v]) and c > 0:
            d = dfs(v, t, min(f, c))
            if d > 0:
                graph[u][i][1] -= d
                graph[v][r][1] += d
                return d

    return 0

In [7]:
graph = gen_graph()
s = 0
t = N
flow = 0

while True:
    used = [False] * (N + 1)
    f = dfs(s, t, float('inf'))
    if f == 0:
        break
    flow += f
    
print(flow)

1


# GRL_7_A

二部グラフマッチング

http://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=GRL_7_A&lang=jp

In [8]:
X, Y, E = 3, 4, 6
edges = [[0, 0], [0, 2], [0, 3], [1, 1], [2, 1], [2, 3]]

print('ans: 3')

ans: 3


In [12]:
def gen_graph(s, t):
    graph = [[] for _ in range(X + Y + 2)]
    
    def add_edges(u, v, c):
        graph[u].append([v, c, len(graph[v])])
        graph[v].append([u, 0, len(graph[u]) - 1])
    
    for i in range(X):
        add_edges(s, i, 1)
        
    for i in range(Y):
        add_edges(i + X, t, 1)
    
    for u, v in edges:
        add_edges(u, v + X, 1)

    return graph

In [10]:
def dfs(u, t, f):
    if u == t:
        return f
    
    used[u] = True
    for i in range(len(graph[u])):
        v, c, r = graph[u][i]
        if (not used[v]) and (c > 0):
            d = dfs(v, t, min(f, c))
            if d > 0:
                graph[u][i][1] -= d
                graph[v][r][1] += d
                return d

    return 0

In [11]:
s = X + Y
t = s + 1
graph = gen_graph(s, t)
flow = 0

while True:
    used = [False] * (X + Y + 2)
    f = dfs(s, t, float('inf'))
    flow += f
    if f == 0:
        break

print(flow)

3


### よりシンプルな解法

In [20]:
def gen_graph():
    graph = [[] for _ in range(X + Y)]
    
    for u, v in edges:
        graph[u].append(v + X)
        graph[v + X].append(u)

    return graph

In [21]:
def dfs(u):
    used[u] = True
    for v in graph[u]:
        w = match[v]
        if (w < 0) or (not visited[w]) and dfs(w):
            match[u] = v
            match[v] = u
            return True
        
    return False

In [22]:
ans = 0
graph = gen_graph()
match = [-1] * (X + Y)

for v in range(X + Y):
    if match[v] < 0:
        visited = [False] * (X + Y)
        if dfs(v):
            ans += 1
            
print(ans)

RecursionError: maximum recursion depth exceeded in comparison